![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [40]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


# GO through the dataframe 

In [41]:
cc_apps.info()
# cc_apps[column].unique() to check the column data 

# check the table
# columns 2 and 7 contain the vlaue miss after point
# coulmn 0,1,3 4,5,6 contain the ? in their cells
# coulmn 10 and 12 conatin 0
# coulmn 8 and 9 same
# coulmn 11 three characters
# column 13 conatin + and - 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


## replace the " ? " with nan and then apply dropna

In [42]:
cc_apps.replace('?',np.nan,inplace = True)
cc_apps = cc_apps.dropna()
# cc_apps.info()
# for i in cc_apps:
#     print(i)
#     print(cc_apps[i].unique())
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       659 non-null    object 
 1   1       659 non-null    object 
 2   2       659 non-null    float64
 3   3       659 non-null    object 
 4   4       659 non-null    object 
 5   5       659 non-null    object 
 6   6       659 non-null    object 
 7   7       659 non-null    float64
 8   8       659 non-null    object 
 9   9       659 non-null    object 
 10  10      659 non-null    int64  
 11  11      659 non-null    object 
 12  12      659 non-null    int64  
 13  13      659 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 77.2+ KB


## change the variable to numeric

In [43]:
# cc_apps.to_csv('clean_data.csv',index=False)
# cc_apps = pd.read_csv('clean_data.csv')
# cc_apps.head()
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       659 non-null    object 
 1   1       659 non-null    object 
 2   2       659 non-null    float64
 3   3       659 non-null    object 
 4   4       659 non-null    object 
 5   5       659 non-null    object 
 6   6       659 non-null    object 
 7   7       659 non-null    float64
 8   8       659 non-null    object 
 9   9       659 non-null    object 
 10  10      659 non-null    int64  
 11  11      659 non-null    object 
 12  12      659 non-null    int64  
 13  13      659 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 77.2+ KB


In [44]:
# for column 0
cc_apps[0].unique()
cc_apps[0]=cc_apps[0].map({'a':1,'b':0})
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,1,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,1,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,0,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,0,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [45]:
# for column 2 make it of float type
cc_apps[1] = pd.to_numeric(cc_apps[1], errors='coerce')

In [46]:
# for cloumn 3
cc_apps[3].unique()
cc_apps=pd.get_dummies(cc_apps,columns= [3],drop_first=True)
cc_apps.head()


,0,1,2,4,5,6,7,8,9,10,11,12,13,3_u,3_y
0,0,30.83,0.000,g,w,v,1.25,t,t,1,g,0,+,1,0
1,1,58.67,4.460,g,q,h,3.04,t,t,6,g,560,+,1,0
2,1,24.50,0.500,g,q,h,1.50,t,f,0,g,824,+,1,0
3,0,27.83,1.540,g,w,v,3.75,t,t,5,g,3,+,1,0
4,0,20.17,5.625,g,w,v,1.71,t,f,0,s,0,+,1,0


In [47]:
# for column 4
cc_apps[4].unique()# 
cc_apps=pd.get_dummies(cc_apps,columns= [4],drop_first=True)
cc_apps.head()

,0,1,2,5,6,7,8,9,10,11,12,13,3_u,3_y,4_gg,4_p
0,0,30.83,0.000,w,v,1.25,t,t,1,g,0,+,1,0,0,0
1,1,58.67,4.460,q,h,3.04,t,t,6,g,560,+,1,0,0,0
2,1,24.50,0.500,q,h,1.50,t,f,0,g,824,+,1,0,0,0
3,0,27.83,1.540,w,v,3.75,t,t,5,g,3,+,1,0,0,0
4,0,20.17,5.625,w,v,1.71,t,f,0,s,0,+,1,0,0,0


In [48]:
# for column 5
cc_apps[5].unique()
cc_apps=pd.get_dummies(cc_apps,columns= [5],drop_first=True)
cc_apps.head()

,0,1,2,6,7,8,9,10,11,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x
0,0,30.83,0.000,v,1.25,t,t,1,g,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,58.67,4.460,h,3.04,t,t,6,g,560,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,24.50,0.500,h,1.50,t,f,0,g,824,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,27.83,1.540,v,3.75,t,t,5,g,3,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,20.17,5.625,v,1.71,t,f,0,s,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [49]:
#for column 6
cc_apps[6].unique()
cc_apps=pd.get_dummies(cc_apps,columns= [6],drop_first=True)
cc_apps.head()


,0,1,2,7,8,9,10,11,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z
0,0,30.83,0.000,1.25,t,t,1,g,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,1,58.67,4.460,3.04,t,t,6,g,560,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,1,24.50,0.500,1.50,t,f,0,g,824,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,0,27.83,1.540,3.75,t,t,5,g,3,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,20.17,5.625,1.71,t,f,0,s,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [50]:
cc_apps[8]=cc_apps[8].map({'t':1,'f':0})
cc_apps.head()

,0,1,2,7,8,9,10,11,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z
0,0,30.83,0.000,1.25,1,t,1,g,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,1,58.67,4.460,3.04,1,t,6,g,560,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,1,24.50,0.500,1.50,1,f,0,g,824,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,0,27.83,1.540,3.75,1,t,5,g,3,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,20.17,5.625,1.71,1,f,0,s,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [51]:
cc_apps[9]=cc_apps[9].map({'t':1,'f':0})
cc_apps.head()

,0,1,2,7,8,9,10,11,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z
0,0,30.83,0.000,1.25,1,1,1,g,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,1,58.67,4.460,3.04,1,1,6,g,560,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,1,24.50,0.500,1.50,1,0,0,g,824,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,0,27.83,1.540,3.75,1,1,5,g,3,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,20.17,5.625,1.71,1,0,0,s,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [52]:

# for column 11
cc_apps[11].unique()
cc_apps = pd.get_dummies(cc_apps,columns = [11],drop_first=True)
cc_apps.head()

,0,1,2,7,8,9,10,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,11_p,11_s
0,0,30.83,0.000,1.25,1,1,1,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,1,58.67,4.460,3.04,1,1,6,560,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1,24.50,0.500,1.50,1,0,0,824,+,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,27.83,1.540,3.75,1,1,5,3,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,20.17,5.625,1.71,1,0,0,0,+,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


In [64]:
# for column 13
cc_apps[13].unique()
cc_apps[13]=cc_apps[13].map({'+':1,'-':0})
cc_apps.head()
# cc_apps.info()

,0,1,2,7,8,9,10,12,13,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,11_p,11_s
0,0,30.83,0.000,1.25,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,1,58.67,4.460,3.04,1,1,6,560,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1,24.50,0.500,1.50,1,0,0,824,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,27.83,1.540,3.75,1,1,5,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,20.17,5.625,1.71,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


In [54]:
# cc_apps.to_csv('clean_and_numeric',index=False)

# save that dataframe 

## prepare the model

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV

cc_apps.columns = cc_apps.columns.astype(str)  # Convert all col names to string

x = cc_apps.drop(columns=['13'])  
y = cc_apps['13']  

# STEP 2: Scale the Features


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


# # STEP 3: Split the Data


x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.2, random_state=42
)

# STEP 4: Define the Model and Params


log_reg = LogisticRegression()

# Hyperparameter grid
params = {
    'C': [0.01, 0.1, 1, 10, 100],       # Regularization strength
    'solver': ['liblinear', 'lbfgs'],   # Optimization algorithm
    'max_iter': [100, 200]              # Iteration limits
}

# GridSearchCV to find best combination
clf = GridSearchCV(log_reg, params, cv=5, scoring='accuracy')
clf.fit(x_train, y_train)

# STEP 5: Evaluate the Model


y_pred = clf.predict(x_test)

# Accuracy
best_score = accuracy_score(y_test, y_pred)
print(best_score)

# # Confusion Matrix
# conf_matrix = confusion_matrix(y_test, y_pred)


# # STEP 6: Output Results


# print("✅ Best Parameters from GridSearchCV:", clf.best_params_)
# print("✅ Accuracy on Test Set:", best_score)
# # print("✅ Confusion Matrix:\n", conf_matrix)
# cc_apps.info()

SyntaxError: cannot assign to literal (3688151672.py, line 1)